Aim: Write a program to compute reaching definitions IN, OUT, GEN, KILL from the given input program flow graph.

Input: Program flow graph in the form of matrix (Directed graph). Contents of the blocks.

Output: Computed values of IN, OUT, GEN, KILL iteration wise.

In [1]:
import pandas as pd
from tabulate import tabulate

statements = {
    1: 'b=1',
    2: 'c=2',
    3: 'a=b+c',
    4: 'd=a-b',
    5: 'd=c+d',
    6: 'c=b+c',
    7: 'e=a-b',
    8: 'd=b+c',
    9: 'e=e+1',
    10: 'b=c*d',
    11: 'c=b-d'
}

blocks_data = {
    1: [1, 2],
    2: [3, 4],
    3: [5],
    4: [6, 7],
    5: [8, 9],
    6: [10, 11]
}
graph_data = {
    1: [],
    2: [1, 4],
    3: [2, 5],
    4: [2, 3],
    5: [3],
    6: [4]
}
variables = {}

for i in range(1, len(blocks_data) + 1):
    s = set()
    for x in blocks_data[i]:
        s.add(statements[x][0])
    variables[i] = s

kills = {}
for x in variables:
    s = set()
    for y in statements:
        if y not in blocks_data[x]:
            if statements[y][0] in variables[x]:
                s.add(y)
    kills[x] = s

gens = {}
predecessors = {}

for x in blocks_data:
    gens[x] = set(blocks_data[x])

for x in graph_data:
    predecessors[x] = set(graph_data[x])

IN = {}
OUT = {}

for x in blocks_data:
    IN[x] = set()
    OUT[x] = gens[x]


def display_table(IN, OUT):
    table = [["Block", "GEN", "KILL", "Predecessor", "IN", "OUT"]]
    for i in range(1, len(blocks_data) + 1):
        block = i
        block_gen = gens[i]
        block_kill = kills.get(i, set())
        block_pred = predecessors.get(i, set())
        block_in = IN[i]
        block_out = OUT[i]
        table.append([block, block_gen, block_kill, block_pred, block_in, block_out])
    print(tabulate(table, headers="firstrow"))


change = True
iteration = 0

while change:
    change = False
    iteration += 1
    new_OUT = {}
    for x in blocks_data:
        IN[x] = set()
        for i in predecessors[x]:
            IN[x] = IN[x].union(OUT[i])

        old_OUT = OUT[x]
        new_OUT[x] = gens[x].union(IN[x] - kills.get(x, set()))

        if new_OUT[x] != old_OUT:
            change = True

    OUT = new_OUT
    print("\n")
    print("Iteration", iteration, ":")
    display_table(IN, OUT)




Iteration 1 :
  Block  GEN       KILL         Predecessor    IN            OUT
-------  --------  -----------  -------------  ------------  ------------------
      1  {1, 2}    {10, 11, 6}  set()          set()         {1, 2}
      2  {3, 4}    {8, 5}       {1, 4}         {1, 2, 6, 7}  {1, 2, 3, 4, 6, 7}
      3  {5}       {8, 4}       {2, 5}         {8, 9, 3, 4}  {9, 3, 5}
      4  {6, 7}    {9, 2, 11}   {2, 3}         {3, 4, 5}     {3, 4, 5, 6, 7}
      5  {8, 9}    {4, 5, 7}    {3}            {5}           {8, 9}
      6  {10, 11}  {1, 2, 6}    {4}            {6, 7}        {10, 11, 7}


Iteration 2 :
  Block  GEN       KILL         Predecessor    IN                        OUT
-------  --------  -----------  -------------  ------------------------  ---------------------
      1  {1, 2}    {10, 11, 6}  set()          set()                     {1, 2}
      2  {3, 4}    {8, 5}       {1, 4}         {1, 2, 3, 4, 5, 6, 7}     {1, 2, 3, 4, 6, 7}
      3  {5}       {8, 4}       {2, 5}    